In [7]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
from tqdm import tqdm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

# Pair Programming Decision Tree

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Decision Tree en este tipo de problemas. Los objetivos de este pair programming :
- Ajustad un modelo de Decision Tree a nuestros datos.
- Calculad las métricas a nuestro nuevo modelo.
- Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [2]:
df = pd.read_pickle('datos/Invistico_Airline_v4_enco.pkl')

In [3]:
df.head(2)

,satisfaction,Gender,Customer Type,Type of Travel,Class,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,...,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes
0,1,1,2,1.0,1.00,0,0,0,2,2,...,3,0,3,5,3,2,1.691495,-1.671090,-0.386036,-0.392329
1,1,0,2,1.0,1.82,0,0,0,3,0,...,4,4,4,2,3,2,0.500825,0.470348,7.786328,7.536854


In [9]:
# separamos los datos en X e y

X = df.drop("satisfaction", axis = 1)
y = df["satisfaction"]

In [10]:
# dividimos nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 37)

In [11]:
# creamos el objeto del modelo
arbol = DecisionTreeClassifier(random_state =0)

# ajustamos el modelo
arbol.fit(x_train, y_train)

DecisionTreeClassifier(random_state=0)

In [12]:
max_features = np.sqrt(len(x_train.columns))
max_features

4.69041575982343

In [13]:
print(arbol.tree_.max_depth)

36


In [16]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test = arbol.predict(x_test)
y_pred_train = arbol.predict(x_train)

In [15]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    
    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [17]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test, y_pred_test, y_train, y_pred_train, "Decision Tree I")
dt_results1

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.939069,0.94159,0.947055,0.944315,0.877047,test,Decision Tree I
1,1.000000,1.00000,1.000000,1.000000,1.000000,train,Decision Tree I


In [18]:
param = {"max_depth": [4, 10, 16, 22, 28, 32], 
        "max_features": [2,3,4,5],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [19]:
gs = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 37), 
            param_grid= param, 
            cv=10,  
            verbose=-1) 

In [20]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=37),
             param_grid={'max_depth': [4, 10, 16, 22, 28, 32],
                         'max_features': [2, 3, 4, 5],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             verbose=-1)

In [21]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeClassifier(max_depth=16, max_features=5, min_samples_leaf=10,
                       min_samples_split=10, random_state=37)

# Pair Programming Random Forest

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Random Forest en este tipo de problemas. Los objetivos de este pair programming :
- Ajustad un modelo de Random Forest a nuestros datos.
- Calculad las métricas a nuestro nuevo modelo.
- Comparad las métricas con los modelos hechos hasta ahora. ¿Cuál es mejor?